# Regression using satellite images

In [12]:
import sklearn.preprocessing as preprocessing
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
from sklearn.metrics import r2_score

In [13]:
class BiodiversityDataset(Dataset):

    def __init__(self, csv_file, img_dir, image_transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with features.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a image.
        """
        self.features = pd.read_csv(csv_file, index_col=['longitude', 'latitude'])
        self.img_dir = img_dir
        self.image_transform = image_transform
        
    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        img_name = str(idx[0]) + "_" + str(idx[1]) + ".jpg"
        img_path = os.path.join(self.img_dir,
                                img_name)
        
        image = Image.open(img_path)
        if self.image_transform:
            image = self.image_transform(image)

        out = torch.from_numpy(np.array([self.features.loc[idx]['habitat_richness']])).detach().clone().reshape([-1, 1])
        features = self.features.drop(columns=['habitat_richness']).loc[idx].values
        features = torch.from_numpy(features.astype('float').reshape(-1, 46)).detach().clone()
        sample = {'image': image, 'features': features, 'out': out}

        return sample
    
    def get_feature(self):
         return torch.from_numpy(self.features.drop(columns=['habitat_richness']).values).detach().clone()
    
    def get_out(self):
        return torch.from_numpy(self.features['habitat_richness'].values).detach().clone().reshape(-1, 1)
    
    def get_index(self):
        return dataset.features.index

In [14]:
class FeatureImageNet(nn.Module):

    def __init__(self, image_net, feature_input_size, output_size, feature_scaler):
        super(FeatureImageNet, self).__init__()
        self.image_net = image_net
        self.feature_scaler = feature_scaler
    
        self.feat_fc1 = nn.Linear(feature_input_size, 46)  
        self.feat_fc2 = nn.Linear(46, 100)
        self.feat_fc3 = nn.Linear(100, 128)

        self.fc1 = nn.Linear(256, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, images, features):
        x_features = self.feature_scaler.transform(features)
        x_features = F.relu(self.feat_fc1(x_features))
        x_features = F.relu(self.feat_fc2(x_features))
        x_features = F.relu(self.feat_fc3(x_features))
        
        x_image = self.image_net(images)
      
        x = torch.cat([x_features, x_image] , dim=1, out=None)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x


In [15]:
class PyTMinMaxScaler():
    """
    Transforms each channel to the range [0, 1].
    """
    def transform(self, tensor):
        tensor.mul_(self.scale).sub_(self.subtract)
        return tensor.float() 
    
    def fit(self, tensor):
        t = tensor.detach().clone()
        self.scale = 1.0 / (t.max(dim=0, keepdim=True)[0] - t.min(dim=0, keepdim=True)[0])
        self.scale[self.scale == float("Inf")] = 0
        t.mul_(self.scale)
        self.subtract = t.min(dim=0, keepdim=True)[0]
        
    def fit_transform(self, tensor):
        self.scale = 1.0 / (tensor.max(dim=0, keepdim=True)[0] - tensor.min(dim=0, keepdim=True)[0])
        self.scale[self.scale == float("Inf")] = 0
        tensor.mul_(self.scale)
        self.subtract = tensor.min(dim=0, keepdim=True)[0]
        tensor.sub_(self.subtract)
        return tensor.float() 
        

In [16]:
data_dir = "../Dataset/images"

image_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

feature_scaler = PyTMinMaxScaler()
out_scaler = PyTMinMaxScaler()

dataset = BiodiversityDataset(data_dir+"/test.csv", data_dir+"/test", image_transform)

feature_scaler.fit(dataset.get_feature())
out_scaler.fit(dataset.get_out())


resnet = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)
resnet.fc = nn.Linear(512, 128)

net = FeatureImageNet(image_net=resnet, feature_input_size=46, output_size=1, feature_scaler=feature_scaler)

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)


Using cache found in C:\Users\minoc/.cache\torch\hub\pytorch_vision_v0.6.0


## Train

In [17]:
def train_model(dataset, model, optimizer, criterion, epochs=20, print_step=1000):
    for epoch in range(epochs): 

        running_loss = 0.0
        for i, idx in enumerate(dataset.get_index()):
            sample = dataset[idx]
            expected_outputs = out_scaler.transform(sample['out'])
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(sample['image'].unsqueeze(0), sample['features'])
            loss = criterion(outputs, expected_outputs)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % print_step == print_step-1:    # print every 500 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / print_step))
                running_loss = 0.0
    
    print('Finished Training')

In [18]:
train_model(dataset=dataset, model=net, optimizer=optimizer, criterion=criterion, epochs=20, print_step=7)

[1,     7] loss: 0.560
[2,     7] loss: 0.220
[3,     7] loss: 0.222
[4,     7] loss: 0.199
[5,     7] loss: 0.193
[6,     7] loss: 0.204
[7,     7] loss: 0.180
[8,     7] loss: 0.175
[9,     7] loss: 0.144
[10,     7] loss: 0.147
[11,     7] loss: 0.127
[12,     7] loss: 0.132
[13,     7] loss: 0.118
[14,     7] loss: 0.109
[15,     7] loss: 0.079
[16,     7] loss: 0.084
[17,     7] loss: 0.067
[18,     7] loss: 0.041
[19,     7] loss: 0.037
[20,     7] loss: 0.020
Finished Training


## Test

In [19]:
def test_model(dataset, model):
    with torch.no_grad():
        expected_outputs = []
        outputs = []
        for idx in dataset.get_index():
            sample = dataset[idx]
            expected_outputs += out_scaler.transform(sample['out'])
            outputs += net(sample['image'].unsqueeze(0), sample['features'])

    print(f"R2 score: {r2_score(outputs, expected_outputs)}")

In [ ]:
test_model(dataset, net)